# Thermal Preference Elicitation Framework

Currently, this framework has been validated for only 1D (operating temp) feature./

# Step 1

Load required modules

In [1]:
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy
sns.set_context("talk", font_scale = 1.4)
from GPFlowUnimodalPref.GPUnimodalElicit import elicit
from GPFlowUnimodalPref.SynOccupant import datagen

/Users/nimishawalgaonkar/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Step 2

1. Occupant walks inside the room and is exposed to state r.
2. Randomly change the state of the room to new state s.
3. Ask the occupant which state they prefer. 
4. If they say state current state s, then record y as 1. If they say previous state r, then record y as 0.
This first duel is our initial duel.
5. Put the duel value and response in the '../data/duels/duels.csv' file

In [2]:
# Load the duels file
data_file = '../GPFlowUnimodalPref/data/initial_duels/gradtrain1D.npz' # <--- you need to keep on updating this file

In [3]:
data = np.load(data_file)
X = data['X']
Y = data['Y']

In [4]:
iter_num = X.shape[0]

In [9]:
#Tprev = np.array(data.Tprev) # previous state (operating temp.)
#Tcurrent = np.array(data.Tcurrent) # current state <---- elicitation framework will give you this values
#X = np.vstack([[Tprev, Tcurrent]]).T
#X = X.astype(float) # features need to be float
X_prime = np.linspace(20, 30, 10)[:,None]
#Y = np.array(data.y)[:,None] # response of the occupant <---- you need to ask occupant about this

In [10]:
Y.shape

(40,)

# Step 3

Now, we want to find the next elicited state. So, we need to use **elicit** module.

In [11]:
config_file = '../GPFlowUnimodalPref/config_files/thermal_config.json' # configuration for grid (how fine you want to be)
trial_num = 10 # this is just to save all the plots in '../data/results/T1' if trial_num = 1 ; T2 if trial_num = 2
model_num = 2
mcmc = True
reachable = True
savefig = False

In [12]:
Aq =  elicit.IntegratedAquisition(X, Y[:,None], X_prime, config_file, model_num, mcmc, reachable)

----------------------------------------
Model is 2
----------------------------------------
burn-in sampling started
Iteration:  100 	 Acc Rate:  0.0 %
Iteration:  200 	 Acc Rate:  0.0 %


KeyboardInterrupt: 

In [ ]:
next_state, next_duel, meanexp, max_exp_imp = Aq.EUI(iter_num, trial_num, savefig)

## Step 3 is the most important step. It outputs next state.

# next_state ?

In [ ]:
next_state

# next duel ?

One of the state is always shared between two duels. So, this is nothing but concatenation of next_state with previous.

In [ ]:
next_duel

# max EUI ?

In [ ]:
max_exp_imp

# Step 4 (Sanity checks)

Is our framework on the right track?

Check -
1. Max Expected Improvement value (is it less than the previous iteration's expected improvement? If so, GOOD!
2. Check the Expected Improvement plots. This will be saved automatically in '../data/results/T1/exp_imp_plots/iteration_num'.
3. Also check utility samples, how they look? Do they make sense? Is our framework going towards max? Some utility samples will also be saved in '../data/results/T1/utility_samples/iteration_num'.

# Step 5

1. Once you have verified that the framework is on the right track, change the operating temp of the room to the next_state as above. 
2. Its fine if you are not able to acheive the next state accurately, just record the measured next state value.
2. Ask the occupant again, which state does he prefer.
3. Add the new measured state and response to the csv  '../data/duels/duels.csv' file.
4. Add max_exp_imp to the MEUI column.
5. Run the notebook again with updated csv file.
6. As you progress with the elicitation, you will notice that the ratio $(MEUI_{(i+1)} - MEUI_{(i)})/MEUI_{(i)}$ will decrease. Based on pilot study, stop the elicitation, once the ratio becomes small enough.

In [ ]:
V = datagen.ThermalPrefDataGen(config_file)
Ynew = V.response_gen1D(next_duel[:,None].T)
Ynew

In [ ]:
Aq.m

In [ ]:
model = Aq.m
samples = Aq.samples

In [ ]:
sample_df = model.get_samples_df(samples)

In [ ]:
xnew = np.linspace(20,27,20)[:,None]
xx = V.normalize1D(xnew)
meanmat = np.zeros(shape = (samples.shape[0], xx.shape[0]))
varmat = np.zeros(shape = (samples.shape[0], xx.shape[0]))
for i, s in sample_df.iterrows():
    model.set_parameter_dict(s)
    mean, v = model.predict_f(xx)
    var = v[:,:,0]
    meanmat[i,:] = mean[:,0]
    varmat[i,:] = np.diag(var)

In [ ]:
def visualize_utility(Xgrid, Mgrid, Vargrid):
    """
    Visualize 1D utility funciton values
    Xgrid : grid states
    Mgrid : mean of GP at those finite grid points
    Vargrid : variance of GP at those finite grid points
    """
    Stdgrid = np.sqrt(Vargrid)
    lower = Mgrid - 2*Stdgrid
    upper = Mgrid + 2*Stdgrid
    #plt.figure(figsize=(12,8))
    #plt.plot(Xgrid[:,0], lower, 'g')
    #plt.plot(Xgrid[:,0], upper, 'r')
    #plt.plot(Xgrid[:,0], Mgrid, 'b')
    
    line, = plt.plot(Xgrid, Mgrid, lw = 2, color = 'b', label = 'utility', alpha = 0.5)
    plt.fill_between(Xgrid[:,0], lower, upper,
                     color = line.get_color(), alpha = 0.25)
    plt.xlabel('Temperature degC')
    plt.ylabel('Utility')
    plt.title('Utility at different temp values')
    return

In [ ]:
mini = 10
maxi = 60
plt.figure(figsize=(12,8))
for i in xrange(mini,maxi):
    visualize_utility(xnew, meanmat[i,:], varmat[i,:])
#plt.legend(loc = 'best')

In [ ]:
model

In [ ]:
from scipy.stats import norm
def diff_norm_g(m, samples, Xgridnorm, Xgrid, mini, maxi):
    """
    Different utilities along with the associated uncertainities
    """
    plt.figure(figsize=(12,8))
    for i in xrange(mini,maxi):
        m.set_state(samples[i,:])
        g = m.predict_g_samples(Xgridnorm, 1)
        plt.plot(Xgrid, norm.cdf(g[0,:,:]), 'b', lw=2, alpha = 0.25)
    a = np.linspace(19.8, 27.2, 100)
    plt.plot(a, 0.5*np.ones(a.shape[0]), 'k')
    plt.xlim(19.8,27.2)
    plt.xlabel('Temperature degC')
    plt.ylabel('Indicator')
    plt.title('Indicator at different temp values')

In [ ]:
diff_norm_g(model, samples, xx, xnew, 0, 5000)

In [ ]:
from scipy.stats import norm
def diff_g(m, samples, Xgridnorm, Xgrid, mini, maxi):
    """
    Different utilities along with the associated uncertainities
    """
    plt.figure(figsize=(12,8))
    for i in xrange(mini,maxi):
        m.set_state(samples[i,:])
        g = m.predict_g_samples(Xgridnorm, 1)
        plt.plot(Xgrid, g[0,:,:], 'b', lw=2, alpha = 0.25)
    a = np.linspace(19.8, 27.2, 100)
    plt.plot(a, np.zeros(a.shape[0]), 'k')
    plt.xlabel('Temperature degC')
    plt.ylabel('Latent g')
    plt.title('Latent g at different temp values')
    plt.xlim(19.8,27.2)

In [ ]:
diff_g(model, samples, xx, xnew, 0, 2600)

In [ ]:
model.set_state(samples[2])
model

In [ ]:
sample_df = model.get_samples_df(samples)
sample_df.head()

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(sample_df['unimodal_model.kern_f.lengthscale'])
plt.plot(sample_df['unimodal_model.kern_f.signal_variance'])

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(sample_df['unimodal_model.kern_f.lengthscale'],
            sample_df['unimodal_model.kern_f.signal_variance'], 'k.', alpha = 0.15)
plt.xlabel('signal_lengthscale')
plt.ylabel('signal_variance')

In [ ]:
samples.shape

In [ ]:
Aq.X.shape

In [ ]:
model.X_concat.value.shape

In [ ]:
a = model.X.value
b = model.X_prime

In [ ]:
a[a.shape[0]/2:,:].shape

In [ ]:
a = np.array([0.4, -3, 2, 6])

In [ ]:
a[a > 0] = 1
a[a < 0] = -1

In [ ]:
a

In [ ]:
ind*-1